## Description
_______

This script calculates statistics for the simulated vs. observed streamflow from the ESP outputs. 
Statistics currently being calculated are bias for each year included in the ESP analysis, correlation coefficient,
RMSE and NSE (Huang et al. 2017). These statistics are being calculated with the mean of the ensemble. 
The variables used for the calculations are described in the "Other Stitistics" cell so that additional stats can easily be added. Stats are calculated between start_date and end_date

### Import Libraries

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from scipy.integrate import simps

### Inputs

In [2]:
# Define inputs for the plot
directory_path= '../58213_esp_results/' # directory containing ESP outputs
start_date= '04-01'       # start date for esp analysis in %Y-%m-%d
end_date= '04-30'         # end date for esp analysis in %Y-%m-%d
output_directory= '../'   # location for the outputs

In [3]:
# adding computed runoff for given year (optional, replace with False if not using)
computed_path= None #'../0058213.txt'
swe_rank= '../swe_analysis/swerank.csv' 

### Generate Plot

In [4]:
# Extract month and day from the dates
start_month, start_day = start_date.split('-')
end_month, end_day = end_date.split('-')

# Convert dates to integers
start_month, start_day = int(start_month), int(start_day)
end_month, end_day = int(end_month), int(end_day)

In [5]:
# Initialize an empty list to store the total simulated runoff for each ensemble member
all_sum_cout_series = []

# Initialize an empty list to store the total observed runoff for each ensemble member
all_sum_rout_series = []

In [6]:
if computed_path is not None:
    # Read the DataFrame from the computed_path
    sim = pd.read_csv(computed_path, sep='\t', index_col=0)

    sim = sim.drop('UNITS', axis=0)

    # Convert index to datetime format
    sim.index = pd.to_datetime(sim.index, errors='coerce')

    # Convert 'cout' column to numeric if needed
    sim['cout'] = pd.to_numeric(sim['cout'], errors='coerce')
    
    # Extract unique years from the index of sim
    unique_years = sim.index.year.unique()

    # Initialize an empty list to store trimmed DataFrames
    trimmed_dfs = []

    # Loop through each unique year
    for year in unique_years:
        # Trim the DataFrame to keep only data within the specified start and end dates for each year
        year_start_date = '{}-{}'.format(year, start_date)
        year_end_date = '{}-{}'.format(year, end_date)
        trimmed_df = sim.loc[year_start_date:year_end_date]
        trimmed_dfs.append(trimmed_df)

    # Concatenate the trimmed DataFrames into a single DataFrame
    sim_trimmed = pd.concat(trimmed_dfs)

    # Drop all columns except for 'cout'
    sim_cout = sim_trimmed[['cout']]


In [7]:
if computed_path is not None:
    # Read the DataFrame from the computed_path
    sim = pd.read_csv(computed_path, sep='\t', index_col=0)

    sim = sim.drop('UNITS', axis=0)

    # Convert index to datetime format
    sim.index = pd.to_datetime(sim.index, errors='coerce')

    # Convert 'cout' column to numeric if needed
    sim['cout'] = pd.to_numeric(sim['cout'], errors='coerce')
    
    # Extract unique years from the index of sim
    unique_years = sim.index.year.unique()

    # Initialize an empty list to store trimmed DataFrames
    trimmed_dfs = []

    # Loop through each unique year
    for year in unique_years:
        # Trim the DataFrame to keep only data within the specified start and end dates for each year
        year_start_date = '{}-{}'.format(year, start_date)
        year_end_date = '{}-{}'.format(year, end_date)
        trimmed_df = sim.loc[year_start_date:year_end_date]
        trimmed_dfs.append(trimmed_df)

    # Concatenate the trimmed DataFrames into a single DataFrame
    sim_trimmed = pd.concat(trimmed_dfs)

    # Drop all columns except for 'cout'
    sim_cout = sim_trimmed[['cout']]

In [8]:
# Iterate through each .nc file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.nc'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Open the esp output file
        esp = xr.open_dataset(file_path)
        
        # Convert all data variable values to float
        esp = esp.astype(float)
        
        # Convert 'DATE' coordinate to datetime format
        esp['DATE'] = pd.to_datetime(esp['DATE'])
        
        # Extract the year from the last DATE
        last_date_year = pd.to_datetime(esp['DATE'][-1].values).year
        
        # Create start_date and end_date for the analysis period
        start_date = pd.Timestamp(year=last_date_year, month=start_month, day=start_day)
        end_date = pd.Timestamp(year=last_date_year, month=end_month, day=end_day)
        
        # Select data between start_date and end_date 
        ds_selected = esp.sel(DATE=slice(start_date, end_date))
        
        # Sum 'cout' variable for each ensemble member
        sum_cout = ds_selected['cout'].mean(dim='ensemble_member')
        sum_rout= ds_selected['rout'].mean(dim='ensemble_member')
        
        # Convert sum_cout to pandas Series
        sum_cout_series = sum_cout.to_series()
        sum_rout_series = sum_rout.to_series()
        
        # Ensure the simulated index is a DatetimeIndex and add year of analysis to series
        sum_cout_series.index = pd.to_datetime(sum_cout_series.index)
        sum_cout_series.index = sum_cout_series.index.map(lambda x: x.replace(year=last_date_year))
        
        # Ensure the observed index is a DatetimeIndex and add year of analysis to series
        sum_rout_series.index = pd.to_datetime(sum_rout_series.index)
        sum_rout_series.index = sum_rout_series.index.map(lambda x: x.replace(year=last_date_year))
        
        # Append the simulated and observed series to the list
        all_sum_cout_series.append(sum_cout_series)
        all_sum_rout_series.append(sum_rout_series)


In [9]:
# Concatenate all sum_cout_series into a single series
sum_cout_series_combined = pd.concat(all_sum_cout_series)
sum_rout_series_combined = pd.concat(all_sum_rout_series)

#### Calculate Statistics

In [10]:
# Create results dataframe
results = pd.DataFrame({
    'ESP Mean Total Flow (cms)': sum_cout_series_combined,
    'Observed Total Flow (cms)': sum_rout_series_combined
})

In [11]:
# Calculate the percent bias for each day
results['Percent Bias'] = ((results['ESP Mean Total Flow (cms)'] - results['Observed Total Flow (cms)']) / results['Observed Total Flow (cms)']) * 100

# Group by year and calculate the mean percent bias for each year
mean_percent_bias_by_year = results.groupby(results.index.year)['Percent Bias'].mean()

In [12]:
# Create a DataFrame called 'statistics' from the Series with index defined
statistics = pd.DataFrame(columns=['Mean Percent Bias'], index=mean_percent_bias_by_year.index)

In [13]:
# Populate the 'Mean Percent Bias' column with the values from 'mean_percent_bias_by_year'
statistics['Mean Percent Bias'] = mean_percent_bias_by_year.values

In [14]:
# Initialize an empty list to store correlation coefficients
correlation_coefficients = []

# Iterate over unique years in the index
for year in results.index.year.unique():
    # Filter the data for the current year
    year_data = results[results.index.year == year]
    # Calculate the correlation coefficient for the current year
    correlation_coefficient = np.corrcoef(year_data['ESP Mean Total Flow (cms)'], year_data['Observed Total Flow (cms)'])[0, 1]
    # Append the correlation coefficient to the list
    correlation_coefficients.append(correlation_coefficient)

statistics['Correlation Coefficient'] = correlation_coefficients
    

In [15]:
# Initialize an empty list to store RMSE values
rmse_values = []

# Iterate over unique years in the index
for year in results.index.year.unique():
    # Filter the data for the current year
    year_data = results[results.index.year == year]
    # Calculate RMSE for the current year
    rmse = np.sqrt(np.mean((year_data['Observed Total Flow (cms)'] - year_data['ESP Mean Total Flow (cms)'])**2))
    # Append the RMSE value to the list
    rmse_values.append(rmse)

# Create a DataFrame called 'statistics' with 'RMSE' for each year
statistics['RMSE'] = rmse_values

In [16]:
# Initialize an empty list to store NSE values
nse_values = []

# Iterate over unique years in the index
for year in results.index.year.unique():
    # Filter the data for the current year
    year_data = results[results.index.year == year]
    # Calculate the mean of observed flow for the current year
    mean_observed = year_data['Observed Total Flow (cms)'].mean()
    # Calculate the numerator of the Nash-Sutcliffe Efficiency (NSE)
    numerator = np.sum((year_data['Observed Total Flow (cms)'] - year_data['ESP Mean Total Flow (cms)'])**2)
    # Calculate the denominator of the Nash-Sutcliffe Efficiency (NSE)
    denominator = np.sum((year_data['Observed Total Flow (cms)'] - mean_observed)**2)
    # Calculate NSE for the current year
    nse = 1 - (numerator / denominator)
    # Append the NSE value to the list
    nse_values.append(nse)

# Create a DataFrame called 'statistics' with 'NSE' for each year
statistics['NSE'] = nse_values

SWE Rank

In [17]:
if swe_rank is not None:
    # Read CSV into a DataFrame
    swe_rank_df = pd.read_csv(swe_rank, index_col=0)
    
    # Merge 'SWE_Rank' column onto 'statistics' based on indexes
    statistics= pd.merge(statistics, swe_rank_df['SWE_Rank'], left_index=True, right_index=True)
    
    
    # Move 'SWE_Rank' column to the first position
    statistics.insert(0, 'SWE_Rank', statistics.pop('SWE_Rank'))


In [18]:
if computed_path is not None:
    
    # Extract unique years from the index of sum_cout_series_combined
    unique_years_sum = sum_cout_series_combined.index.year.unique()

    # Filter sim_cout to include only the years found in sum_cout_series_combined
    sim_cout_filtered = sim_cout[sim_cout.index.year.isin(unique_years_sum)].copy()
    
    # Assign values from sum_cout_series_combined to a new column 'ESP Mean Total Flow (cms)'
    sim_cout_filtered.loc[:, 'ESP Mean Total Flow (cms)'] = sum_cout_series_combined.values

    # Calculate the percent bias for each day
    sim_cout_filtered['Percent Bias'] = ((sim_cout_filtered['ESP Mean Total Flow (cms)'] - sim_cout_filtered['cout']) / sim_cout_filtered['cout']) * 100

    # Group by year and calculate the mean percent bias for each year
    mean_percent_bias_by_year_cout = sim_cout_filtered.groupby(sim_cout_filtered.index.year)['Percent Bias'].mean()
    
    # Create a DataFrame called 'statistics' from the Series with index defined
    cout_stats = pd.DataFrame(columns=['Mean Percent Bias'], index=mean_percent_bias_by_year.index)
    
    # Populate the 'Mean Percent Bias' column with the values from 'mean_percent_bias_by_year'
    cout_stats['Mean Percent Bias'] = mean_percent_bias_by_year_cout.values
    
    # Initialize an empty list to store correlation coefficients
    correlation_coefficients1 = []

    # Iterate over unique years in the index
    for year in sim_cout_filtered.index.year.unique():
        # Filter the data for the current year
        year_data1 = sim_cout_filtered[sim_cout_filtered.index.year == year]
        # Calculate the correlation coefficient for the current year
        correlation_coefficient1 = np.corrcoef(year_data1['ESP Mean Total Flow (cms)'], year_data1['cout'])[0, 1]
        # Append the correlation coefficient to the list
        correlation_coefficients1.append(correlation_coefficient1)

    cout_stats['Correlation Coefficient'] = correlation_coefficients1
    
    
        # Initialize an empty list to store RMSE values
    rmse_values1 = []

    # Iterate over unique years in the index
    for year in sim_cout_filtered.index.year.unique():
        # Filter the data for the current year
        year_data = sim_cout_filtered[sim_cout_filtered.index.year == year]
        # Calculate RMSE for the current year
        rmse1 = np.sqrt(np.mean((year_data['cout'] - year_data['ESP Mean Total Flow (cms)'])**2))
        # Append the RMSE value to the list
        rmse_values1.append(rmse1)

    # Create a DataFrame called 'statistics' with 'RMSE' for each year
    cout_stats['RMSE'] = rmse_values1
    
    # Initialize an empty list to store NSE values
    nse_values1= []

    # Iterate over unique years in the index
    for year in sim_cout_filtered.index.year.unique():
        # Filter the data for the current year
        year_data = sim_cout_filtered[sim_cout_filtered.index.year == year]
        # Calculate the mean of observed flow for the current year
        mean_observed1 = year_data['cout'].mean()
        # Calculate the numerator of the Nash-Sutcliffe Efficiency (NSE)
        numerator1 = np.sum((year_data['cout'] - year_data['ESP Mean Total Flow (cms)'])**2)
        # Calculate the denominator of the Nash-Sutcliffe Efficiency (NSE)
        denominator1 = np.sum((year_data['cout'] - mean_observed1)**2)
        # Calculate NSE for the current year
        nse1 = 1 - (numerator1 / denominator1)
        # Append the NSE value to the list
        nse_values1.append(nse1)

    # Create a DataFrame called 'statistics' with 'NSE' for each year
    cout_stats['NSE'] = nse_values1
    
    if swe_rank is not None:
        # Read CSV into a DataFrame
        swe_rank_df = pd.read_csv(swe_rank, index_col=0)

        # Merge 'SWE_Rank' column onto 'statistics' based on indexes
        cout_stats= pd.merge(cout_stats, swe_rank_df['SWE_Rank'], left_index=True, right_index=True)

        # Move 'SWE_Rank' column to the first position
        cout_stats.insert(0, 'SWE_Rank', cout_stats.pop('SWE_Rank'))
        
    
    # define output file
    cout_statistics_filename= 'simulated_esp_stats.csv'
    cout_stats_output_path = output_directory + cout_statistics_filename
    
    # Round all values to two decimal places
    cout_stats = cout_stats.round(2)
    
    # Save bias to CSV
    cout_stats.to_csv(cout_stats_output_path)


#### Outputs

In [19]:
# define output file
statistics_filename= 'esp_stats.csv'
stats_output_path = output_directory + statistics_filename

In [20]:
# Convert all values in DataFrames to floats
#statistics = results.astype(float)

# Round all values to two decimal places
statistics = statistics.round(2)

In [21]:
# Save bias to CSV
statistics.to_csv(stats_output_path)